In [ ]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
import tensorflow as tf
import numpy as np

#ハイパーパラメータの設定
batch_size = 100 #バッチサイズ
original_dim = 784　#入力データ次元
latent_dim = 2 #潜在空間次元
intermediate_dim = 256 #中間層次元
epochs = 10 #エポック数
epsilon_std = 1.0 #

In [24]:
#エンコーダの作成

def sampling(args: tuple):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0],latent_dim),
                              mean = 0.,
                              stddev = epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon


x = Input(shape = (original_dim,), name = "input")
h = Dense(intermediate_dim, activation = "relu", name = "encoding")(x)
z_mean = Dense(latent_dim, name = "mean")(h)
z_log_var = Dense(latent_dim, name = "log-variance")(h)
z = Lambda(sampling, output_shape = (latent_dim,))([z_mean, z_log_var])
encoder = Model(x, [z_mean, z_log_var, z], name = "encoder")

encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 784)]        0                                            
__________________________________________________________________________________________________
encoding (Dense)                (None, 256)          200960      input[0][0]                      
__________________________________________________________________________________________________
mean (Dense)                    (None, 2)            514         encoding[0][0]                   
__________________________________________________________________________________________________
log-variance (Dense)            (None, 2)            514         encoding[0][0]                   
____________________________________________________________________________________________

In [26]:
#デコーダの作成
input_decoder = Input(shape = (latent_dim,), name = "decoder_input")
decoder_h = Dense(intermediate_dim,
                  activation = "relu",
                  name = "decoder_h")(input_decoder)
x_decoded = Dense(original_dim, 
                  activation = "sigmoid",
                  name = "flat_decoded")(decoder_h)
decoder = Model(input_decoder, x_decoded, name = "decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
decoder_h (Dense)            (None, 256)               768       
_________________________________________________________________
flat_decoded (Dense)         (None, 784)               201488    
Total params: 202,256
Trainable params: 202,256
Non-trainable params: 0
_________________________________________________________________


In [27]:
#モデルをまとめる
output_combined = decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
encoder (Functional)         [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Functional)         (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [28]:
#損失関数の定義
def vae_loss(x: tf.Tensor,
             x_decoded_mean: tf.Tensor,
             z_log_var = z_log_var,
             z_mean = z_mean,
             original_dim = original_dim):
    
    xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
    kl_loss = -0.5 * K.sum(
        1 + z_log_var - K.square(z_mean) - K.exp(z_log_var),
        axis = -1)
    vae_loss = K.mean(xent_loss + kl_loss)
    return vae_loss

#モデルのコンパイル
vae.compile(optimizer = "rmsprop", loss = vae_loss, metrics = ["accuracy"])
vae.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
encoder (Functional)         [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Functional)         (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [30]:
#データの準備
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
x_test = (x_test.reshape(-1, 784) / 255).astype(np.float32)

In [22]:
vae.fit(x_train, x_train, 
        shuffle = True,
        epochs = epochs,
        batch_size = batch_size)

Epoch 1/10


NameError: in user code:

    /Users/hokudai/Desktop/workspace/deepenv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-20-a623d50a26af>:61 vae_loss  *
        xent_loss = original_dim * metrics.binary_crossentropy(x,

    NameError: name 'metrics' is not defined
